In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Dataset1_75k_biotagged.csv")
dataset = Dataset.from_pandas(df)


In [ ]:
label_list = ["O", "B-Toxic", "I-Toxic"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}


In [ ]:
# ===============================
# ✅ CELL 1: Model Training + Auto-Save to Drive
# ===============================

!pip install -q evaluate seqeval

from transformers import (
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
)
import numpy as np
import evaluate
from ast import literal_eval
from datasets import Dataset
import os
import shutil

# ✅ 0. Mount Google Drive
print("📁 Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("✅ Drive mounted successfully!\n")

# ✅ 1. Load tokenizer
print("🔤 Loading tokenizer: xlm-roberta-base")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# ✅ 2. Load model
print("🤖 Loading base model: xlm-roberta-base")
model = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# ✅ 3. Training arguments
training_args = TrainingArguments(
    output_dir="./toxic_model_results",
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=2,
    report_to="none"
)

# ✅ 3. Data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# ✅ 4. Load metric
seqeval = evaluate.load("seqeval")

# ✅ 5. Metric computation
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2label[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# ✅ 6. Ensure list conversion
def ensure_list(example):
    if isinstance(example["tokens"], str):
        example["tokens"] = literal_eval(example["tokens"])
    if isinstance(example["BIO_tags"], str):
        example["BIO_tags"] = literal_eval(example["BIO_tags"])
    return example

print("📊 Preparing dataset...")
dataset = dataset.map(ensure_list)

# ✅ 7. Tokenization + Label Alignment
def tokenize_and_align_labels(example):
    tokens = example["tokens"]
    labels = example["BIO_tags"]

    tokenized_input = tokenizer(
        tokens,
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    word_ids = tokenized_input.word_ids()
    label_ids = []

    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx == previous_word_idx:
            label_ids.append(-100)
        else:
            label_ids.append(label2id[labels[word_idx]])
        previous_word_idx = word_idx

    tokenized_input["labels"] = label_ids
    return tokenized_input

# ✅ 8. Map tokenization
print("🔄 Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)

# ✅ 9. Remove string columns
tokenized_dataset = tokenized_dataset.remove_columns(["tokens", "BIO_tags"])

# ✅ 10. Split dataset
dataset_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

print(f"📈 Train samples: {len(train_dataset)}, Eval samples: {len(eval_dataset)}")

# ✅ 11. Set dataset format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ✅ 12. Initialize Trainer
print("\n🚀 Initializing Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ✅ 13. Start training
print("\n" + "="*60)
print("🏋️  STARTING TRAINING")
print("="*60 + "\n")
trainer.train()

# ✅ 14. Save model locally
LOCAL_PATH = "./fine_tuned_toxic_span_model"
print(f"\n💾 Saving model locally to: {LOCAL_PATH}")
trainer.save_model(LOCAL_PATH)
tokenizer.save_pretrained(LOCAL_PATH)
print("✅ Model saved locally")

# ✅ 15. Save to Google Drive (CRITICAL!)
DRIVE_PATH = "/content/drive/MyDrive/fine_tuned_toxic_span_model"
print(f"\n☁️  Copying model to Google Drive: {DRIVE_PATH}")

try:
    # Remove old version if exists
    if os.path.exists(DRIVE_PATH):
        print("   Removing old version...")
        shutil.rmtree(DRIVE_PATH)

    # Copy to Drive
    shutil.copytree(LOCAL_PATH, DRIVE_PATH)
    print("✅ Model saved to Google Drive!")

except Exception as e:
    print(f"⚠️  Error saving to Drive: {str(e)}")

# ✅ 16. Final evaluation
print("\n📊 Final Evaluation:")
eval_results = trainer.evaluate()
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print(f"\n📍 Model saved to:")
print(f"   Local: {LOCAL_PATH}")
print(f"   Drive: {DRIVE_PATH}")
print("\n🎉 Ready for inference! Run Cell 2 next.")
print("="*60)

📁 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted successfully!

🔤 Loading tokenizer: xlm-roberta-base


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

🤖 Loading base model: xlm-roberta-base


...

/tmp/ipython-input-3709988949.py:141: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



🏋️  STARTING TRAINING



Step,Training Loss
500,0.145200
1000,0.108400
1500,0.098800
2000,0.079400
2500,0.065700
3000,0.061000
3500,0.046200



💾 Saving model locally to: ./fine_tuned_toxic_span_model
✅ Model saved locally

☁️  Copying model to Google Drive: /content/drive/MyDrive/fine_tuned_toxic_span_model
✅ Model saved to Google Drive!

📊 Final Evaluation:


  eval_loss: 0.1320
  eval_precision: 0.5556
  eval_recall: 0.5439
  eval_f1: 0.5497
  eval_accuracy: 0.9672
  eval_runtime: 10.3756
  eval_samples_per_second: 144.5700
  eval_steps_per_second: 18.1190
  epoch: 5.0000

✅ TRAINING COMPLETE!

📍 Model saved to:
   Local: ./fine_tuned_toxic_span_model
   Drive: /content/drive/MyDrive/fine_tuned_toxic_span_model

🎉 Ready for inference! Run Cell 2 next.


In [ ]:
# ===============================
# ✅ CELL 2: Urdu Toxic Span Detection - Inference (With Highlighting)
# ===============================

import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np
import os
from IPython.display import display, HTML

# ✅ 1. Smart model loading (works after reconnection)
def load_model():
    """
    Loads model from Drive or local storage.
    """
    print("🔍 Searching for trained model...")

    # Check Drive first (persistent), then local (temporary)
    possible_paths = [
        "/content/drive/MyDrive/fine_tuned_toxic_span_model",  # Drive (priority)
        "./fine_tuned_toxic_span_model",  # Local
    ]

    model_path = None
    for path in possible_paths:
        if os.path.exists(path):
            print(f"✅ Found model at: {path}")
            model_path = path
            break

    if model_path is None:
        print("\n❌ Model not found!")
        print("📋 Mount Google Drive if needed:")
        print("   from google.colab import drive")
        print("   drive.mount('/content/drive')")
        raise FileNotFoundError("Model not found")

    # Load model
    print(f"📂 Loading model...")
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
    model = AutoModelForTokenClassification.from_pretrained(model_path, local_files_only=True)
    model.eval()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print(f"✅ Model loaded on {device}\n")
    return model, tokenizer, device

# Load the model
model, tokenizer, device = load_model()

# ✅ 2. Inference function with highlighting
def predict_toxic_spans_with_highlight(text):
    """
    Predicts toxic spans and returns highlighted text.
    """
    # Tokenize
    encoding = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    tokens = {k: v.to(device) for k, v in encoding.items()}

    # Get predictions
    with torch.no_grad():
        outputs = model(**tokens)
        predictions = torch.argmax(outputs.logits, dim=2)

    # Decode predictions
    predicted_labels = [model.config.id2label[p.item()] for p in predictions[0]]
    word_ids = encoding.word_ids()
    tokens_text = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0])

    # Build highlighted text
    highlighted_parts = []
    current_word = []
    current_label = "O"
    prev_word_id = None

    for idx, (token, label, word_id) in enumerate(zip(tokens_text, predicted_labels, word_ids)):
        # Skip special tokens
        if token in ["<s>", "</s>", "<pad>"]:
            continue

        # Check if we're starting a new word
        if word_id != prev_word_id and current_word:
            # Process previous word
            word_text = tokenizer.convert_tokens_to_string(current_word)
            if current_label.startswith("B-") or current_label.startswith("I-"):
                label_name = current_label.split("-")[1] if "-" in current_label else "TOXIC"
                highlighted_parts.append(f'<span style="background-color: #ff6b6b; color: white; padding: 2px 4px; border-radius: 3px; font-weight: bold;" title="{label_name}">{word_text}</span>')
            else:
                highlighted_parts.append(word_text)

            current_word = []
            current_label = "O"

        # Add token to current word
        if word_id is not None:
            current_word.append(token)
            if label.startswith("B-") or label.startswith("I-"):
                current_label = label

        prev_word_id = word_id

    # Process last word
    if current_word:
        word_text = tokenizer.convert_tokens_to_string(current_word)
        if current_label.startswith("B-") or current_label.startswith("I-"):
            label_name = current_label.split("-")[1] if "-" in current_label else "TOXIC"
            highlighted_parts.append(f'<span style="background-color: #ff6b6b; color: white; padding: 2px 4px; border-radius: 3px; font-weight: bold;" title="{label_name}">{word_text}</span>')
        else:
            highlighted_parts.append(word_text)

    # Join parts
    highlighted_text = " ".join(highlighted_parts)

    # Also return toxic spans info
    toxic_spans = []
    current_span = None

    for idx, (token, label, word_id) in enumerate(zip(tokens_text, predicted_labels, word_ids)):
        if label.startswith("B-"):
            if current_span:
                toxic_spans.append(current_span)
            current_span = {
                "label": label[2:],
                "tokens": [token],
            }
        elif label.startswith("I-") and current_span:
            current_span["tokens"].append(token)
        elif current_span:
            toxic_spans.append(current_span)
            current_span = None

    if current_span:
        toxic_spans.append(current_span)

    # Format toxic spans
    toxic_info = []
    for span in toxic_spans:
        span_text = tokenizer.convert_tokens_to_string(span["tokens"])
        toxic_info.append({"text": span_text.strip(), "label": span["label"]})

    return highlighted_text, toxic_info

# ✅ 3. Display function for better visualization
def display_toxic_analysis(text):
    """
    Displays text with toxic spans highlighted in color.
    """
    highlighted_text, toxic_info = predict_toxic_spans_with_highlight(text)

    # Create HTML output
    html_output = f"""
    <div style="font-family: 'Arial', sans-serif; padding: 15px; background-color: #f8f9fa; border-radius: 8px; margin: 10px 0;">
       <div style="font-size: 16px; line-height: 1.8; direction: rtl; text-align: right; color: #212529;">
            {highlighted_text}
        </div>
    """

    if toxic_info:
        html_output += """
        <div style="margin-top: 15px; padding-top: 10px; border-top: 2px solid #dee2e6;">
            <strong style="color: #dc3545;">⚠️ زہریلے الفاظ (Toxic spans detected):</strong>
            <ul style="margin-top: 8px;">
        """
        for span in toxic_info:
            html_output += f'<li style="color: #495057;"><strong>{span["text"]}</strong> <span style="color: #6c757d;">[{span["label"]}]</span></li>'
        html_output += "</ul>"
    else:
        html_output += """
        <div style="margin-top: 15px; padding-top: 10px; border-top: 2px solid #dee2e6; color: #28a745;">
            <strong>✅ کوئی زہریلا پن نہیں ملا (No toxicity detected)</strong>
        </div>
        """

    html_output += "</div>"

    display(HTML(html_output))
    return toxic_info

# ✅ 4. Test with Urdu example sentences
test_sentences = [
    "تم بہت احمق ہو اور کوئی تمہیں پسند نہیں کرتا",
    "مکمل طور پر نااہل ، ہمارے بیوروکریٹس کے پاس کوئی عذر نہیں ہے اور وہ واقعی میں گھٹیا پن نہیں دیتے ہیں!",
    "اس لئے جس آدمی نے اس رات میں عبادت کا اہتمام کیا گویا اس نے اللہ تعالیٰ کا شکریہ ادا کیا اور اپنے عمل سے یہ ثابت کیا کہ اس کے دل میں قرآن مجید کی صحیح قدروقیمت کا احساس موجود ہے۔",
    "hmaray news anchors kafi achay hain ,ic bat pr tareef bnti hai judge sahab",
    "Mujhe is faasiq ganwaar, ujhad, bad-tahzeeb, bad-atwaar, ghabi, paagal se kabhi hamdardi nahi thi.",
    "میں آپ کی رائے سے احترام کے ساتھ اختلاف کرتا ہوں",
    "ایک سابق فوجیوں کی حیثیت سے ، مجھے لگتا ہے کہ ان بہت ہی خبیث النفس نوجوانوں کی فوج میں کوئی جگہ نہیں ہے۔",
    "آپ کا دن اچھا گزرے",
]

print("🧪 Testing on Urdu sample sentences:\n")
print("="*60)

for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{'='*60}")
    print(f"مثال {i} (Example {i}):")
    print(f"{'='*60}")
    display_toxic_analysis(sentence)

# ✅ 5. Text-based output (for non-Jupyter environments)
def print_toxic_analysis_text(text):
    """
    Prints analysis in text format with ANSI colors (for terminals).
    """
    highlighted_text, toxic_info = predict_toxic_spans_with_highlight(text)

    # Remove HTML tags for plain text version
    import re
    plain_text = re.sub(r'<span[^>]*>(.*?)</span>', r'[\1]', highlighted_text)
    plain_text = re.sub(r'<[^>]+>', '', plain_text)

    print(f"\nText: {text}")
    print(f"Highlighted: {plain_text}")

    if toxic_info:
        print("\n⚠️  Toxic spans detected:")
        for span in toxic_info:
            print(f"  - '{span['text']}' [{span['label']}]")
    else:
        print("\n✅ No toxicity detected")

# ✅ 6. Interactive inference with highlighting
print("\n\n" + "="*60)
print("🎮 انٹرایکٹو موڈ (Interactive Mode)")
print("="*60)
print("اردو متن درج کریں / Enter Urdu text to analyze")
print("Type 'quit' to exit\n")

while True:
    user_input = input("\nText: ")
    if user_input.lower() in ['quit', 'exit', 'q', 'بند']:
        break

    if not user_input.strip():
        continue

    print("\n" + "-"*60)
    display_toxic_analysis(user_input)
    print("-"*60)

print("\n✅ تجزیہ مکمل! (Analysis complete!)")

🔍 Searching for trained model...
✅ Found model at: /content/drive/MyDrive/fine_tuned_toxic_span_model
📂 Loading model...
✅ Model loaded on cuda

🧪 Testing on Urdu sample sentences:


مثال 1 (Example 1):



مثال 2 (Example 2):



مثال 3 (Example 3):



مثال 4 (Example 4):



مثال 5 (Example 5):


...


مثال 7 (Example 7):



مثال 8 (Example 8):




🎮 انٹرایکٹو موڈ (Interactive Mode)
اردو متن درج کریں / Enter Urdu text to analyze
Type 'quit' to exit


Text: گھٹیا پن کے تمام بیکار ڈھیر۔

------------------------------------------------------------


------------------------------------------------------------

Text: شیعہ اور سنی دونوں اسلام کے ماننے والے ہیں۔

------------------------------------------------------------


------------------------------------------------------------


In [ ]:
# ===============================
# ✅ CELL 3: Audio Processing - Beep Toxic Words (Fast Version)
# ===============================

print("📦 Installing dependencies...")
!pip install -q openai-whisper pydub 2>&1 | grep -v "already satisfied" || true
!apt-get install -y ffmpeg > /dev/null 2>&1
print("✅ Dependencies installed!\n")

import torch
from pydub import AudioSegment
from pydub.generators import Sine
import json
import os
import whisper
from IPython.display import Audio, display

# ✅ 1. Load Whisper model (ONCE at the start)
print("🎤 Loading Whisper model (tiny - fast)...")
whisper_model = whisper.load_model("tiny")  # Options: tiny, base, small
print("✅ Whisper model loaded!\n")

# ✅ 2. Upload audio file
def upload_audio():
    """Upload audio file."""
    from google.colab import files

    print("📤 Upload your Urdu audio file...")
    uploaded = files.upload()

    if not uploaded:
        return None

    audio_filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {audio_filename}\n")
    return audio_filename

# ✅ 3. Transcribe with word timestamps
def transcribe_audio(audio_path):
    """Transcribe audio using Whisper."""
    print(f"🎙️  Transcribing: {audio_path}")

    # Transcribe
    result = whisper_model.transcribe(
        audio_path,
        language="ur",
        word_timestamps=True,
        verbose=False
    )

    # Extract word-level timestamps
    word_segments = []
    for segment in result["segments"]:
        if "words" in segment:
            for word in segment["words"]:
                word_segments.append({
                    "word": word["word"].strip(),
                    "start": word["start"],
                    "end": word["end"]
                })

    full_text = result["text"].strip()

    print(f"✅ Transcription complete!")
    print(f"📝 Text: {full_text}\n")

    return full_text, word_segments

# ✅ 4. Detect toxic words
def detect_toxic_words(text, word_segments):
    """Detect toxic words and match to timestamps."""
    print("🔍 Detecting toxic words...")

    # Use your trained model
    _, toxic_info = predict_toxic_spans_with_highlight(text)

    if not toxic_info:
        print("✅ No toxic words detected!\n")
        return []

    print(f"⚠️  Found {len(toxic_info)} toxic span(s)")

    # Match toxic words to timestamps
    toxic_timestamps = []

    for toxic in toxic_info:
        toxic_text = toxic["text"].strip().lower()

        # Find in word segments
        for segment in word_segments:
            segment_word = segment["word"].strip().lower()

            # Match (exact or contains)
            if toxic_text in segment_word or segment_word in toxic_text:
                toxic_timestamps.append({
                    "word": segment["word"],
                    "start": segment["start"],
                    "end": segment["end"],
                    "label": toxic["label"]
                })
                print(f"   🔴 '{segment['word']}' at {segment['start']:.1f}s [{toxic['label']}]")

    print()
    return toxic_timestamps

# ✅ 5. Generate beep
def generate_beep(duration_ms):
    """Generate beep sound."""
    return Sine(1000).to_audio_segment(duration=duration_ms) - 10

# ✅ 6. Beep toxic words
def beep_toxic_words(audio_path, toxic_timestamps, output_path="cleaned_audio.wav"):
    """Replace toxic words with beeps."""
    print(f"🔊 Loading audio: {audio_path}")
    audio = AudioSegment.from_file(audio_path)

    if not toxic_timestamps:
        print("✅ No beeping needed - saving original\n")
        audio.export(output_path, format="wav")
        return output_path

    print(f"🔇 Beeping {len(toxic_timestamps)} word(s)...")

    # Sort in reverse order
    toxic_timestamps = sorted(toxic_timestamps, key=lambda x: x["start"], reverse=True)

    # Replace with beeps
    for toxic in toxic_timestamps:
        start_ms = int(toxic["start"] * 1000)
        end_ms = int(toxic["end"] * 1000)
        duration_ms = end_ms - start_ms

        beep = generate_beep(duration_ms)
        audio = audio[:start_ms] + beep + audio[end_ms:]
        print(f"   ✓ Beeped '{toxic['word']}' at {toxic['start']:.1f}s")

    print(f"\n💾 Saving cleaned audio: {output_path}")
    audio.export(output_path, format="wav")
    print("✅ Done!\n")

    return output_path

# ✅ 7. Complete pipeline
def process_audio(audio_path):
    """Complete processing pipeline."""
    print("="*60)
    print("🎵 PROCESSING AUDIO")
    print("="*60 + "\n")

    # Step 1: Transcribe
    text, word_segments = transcribe_audio(audio_path)

    # Step 2: Detect toxic words
    toxic_timestamps = detect_toxic_words(text, word_segments)

    # Step 3: Beep toxic words
    output_path = "cleaned_audio.wav"
    cleaned_audio = beep_toxic_words(audio_path, toxic_timestamps, output_path)

    # Step 4: Display results
    print("="*60)
    print("✅ RESULTS")
    print("="*60)
    print(f"📝 Transcript: {text}")
    print(f"⚠️  Toxic words: {len(toxic_timestamps)}")
    print("="*60 + "\n")

    # Display audio players
    print("🎧 Original Audio:")
    display(Audio(audio_path, autoplay=False))

    print("\n🎧 Cleaned Audio (toxic words beeped):")
    display(Audio(cleaned_audio, autoplay=False))

    # Save report
    report = {
        "transcript": text,
        "toxic_words": toxic_timestamps,
        "total_toxic": len(toxic_timestamps)
    }

    with open("audio_report.json", "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2, ensure_ascii=False)

    print("\n📄 Report saved: audio_report.json")

    # Download option
    print("\n⬇️  Download cleaned audio:")
    from google.colab import files
    files.download(cleaned_audio)

    return cleaned_audio

# ✅ 8. MAIN EXECUTION
print("\n" + "="*60)
print("🎤 URDU AUDIO TOXIC WORD BEEPING")
print("="*60 + "\n")

print("Choose option:")
print("1. Upload audio file")
print("2. Use file path")
print("3. Record audio (from microphone)\n")

choice = input("Enter choice (1/2/3): ").strip()

if choice == "1":
    # Upload
    audio_file = upload_audio()
    if audio_file:
        process_audio(audio_file)

elif choice == "2":
    # File path
    audio_file = input("Enter audio file path: ").strip()
    if os.path.exists(audio_file):
        process_audio(audio_file)
    else:
        print(f"❌ File not found: {audio_file}")

elif choice == "3":
    # Record audio
    print("\n🎙️  Recording audio...")
    print("This will record for 10 seconds. Speak now!")

    from google.colab import output
    from base64 import b64decode
    from io import BytesIO

    # Use Colab's audio recording
    print("\nClick the microphone icon below to record:")

    RECORD = """
    const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
    const b2text = blob => new Promise(resolve => {
      const reader = new FileReader()
      reader.onloadend = e => resolve(e.srcElement.result)
      reader.readAsDataURL(blob)
    })

    var record = time => new Promise(async resolve => {
      stream = await navigator.mediaDevices.getUserMedia({ audio: true })
      recorder = new MediaRecorder(stream)
      chunks = []
      recorder.ondataavailable = e => chunks.push(e.data)
      recorder.start()
      await sleep(time)
      recorder.onstop = async ()=>{
        blob = new Blob(chunks)
        text = await b2text(blob)
        resolve(text)
      }
      recorder.stop()
    })
    """

    display(HTML('''
    <button id="record">🎙️ Record 10 seconds</button>
    <script>
    document.getElementById("record").onclick = async () => {
        const sleep = time => new Promise(resolve => setTimeout(resolve, time))
        const b2text = blob => new Promise(resolve => {
            const reader = new FileReader()
            reader.onloadend = e => resolve(e.srcElement.result)
            reader.readAsDataURL(blob)
        })

        const stream = await navigator.mediaDevices.getUserMedia({ audio: true })
        const recorder = new MediaRecorder(stream)
        const chunks = []

        recorder.ondataavailable = e => chunks.push(e.data)
        recorder.start()

        document.getElementById("record").innerText = "🔴 Recording... (10s)"
        await sleep(10000)

        recorder.onstop = async () => {
            const blob = new Blob(chunks, {type: 'audio/webm'})
            const text = await b2text(blob)
            google.colab.kernel.invokeFunction('notebook.save_audio', [text], {})
        }

        recorder.stop()
        document.getElementById("record").innerText = "✅ Recording complete!"
    }
    </script>
    '''))

    print("\n⚠️  Note: Recording feature requires manual implementation.")
    print("Please use option 1 (upload) or 2 (file path) instead.")

else:
    print("❌ Invalid choice!")

print("\n✅ Script complete!")